In [3]:
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, column, desc, window

spark = SparkSession.builder.appName("SparkGuide").master("spark://DOUGPC.:7077").getOrCreate()
print(spark)

your 131072x1 screen size is bogus. expect trouble
22/12/07 21:50:09 WARN Utils: Your hostname, DOUGPC resolves to a loopback address: 127.0.1.1; using 172.22.121.112 instead (on interface eth0)
22/12/07 21:50:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/07 21:50:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
#aqui é a construção de um dataframe reader
flightData15= spark\
    .read\
    .option("inferSchema", True)\
    .option("header", "true")\
    .csv("../Python/SparkDefGuide/d")

AnalysisException: Path does not exist: file:/home/doug/ProjetosEstudo/PySpark/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv

In [ ]:
#utilizando um comando de ação para visualizar o conteúdo do documento
#a função take irá puxar três linhas do DF e mostrar no resultado.
#vale destacar que no fim, essa estrutrua exibida é um array de lista.

flightData15.take(3)


In [ ]:
#entendendo o query plan do spark 
#utilizando a função explain
flightData15.sort("Count", descending=True).explain()

In [ ]:
flightData15.sort("Count", ascending=False).take(1)

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions","5")
flightData15.sort("Count", ascending = False).take(5)

In [ ]:
#configurando a qtd de partições para essa transformação
#essa configuração não manipula a transformação, mas situações como partição e operações de shuffle.
#o que fazemos aqui é dimensionar os recursos para a transformação física do arquivo.

spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData15.sort("Count").explain()

In [5]:
#criando uma table a partir do dataframe utilizado
#daqui, podemos utilizar SparkSQL para consultas

flightData15.createOrReplaceTempView("flight_Data_2015")

NameError: name 'flightData15' is not defined

In [ ]:
sqlWay = spark.sql("""
                   SELECT DEST_COUNTRY_NAME, count(1)
                   FROM FLIGHT_DATA_2015
                   GROUP BY DEST_COUNTRY_NAME""")
dataFrameWay = flightData15\
    .groupBy("DEST_COUNTRY_NAME")\
    .count()
    
sqlWay.explain()
dataFrameWay.explain()


In [ ]:
spark.sql("SELECT MAX(COUNT) FROM FLIGHT_DATA_2015").take(1)

In [ ]:
from pyspark.sql.functions import max

flightData15.select(max("count")).take(1)

In [ ]:
sqlMax = spark.sql("""SELECT
                   DEST_COUNTRY_NAME,
                   SUM(COUNT) AS TOTAL_DESTINATION
                   FROM FLIGHT_DATA_2015
                   GROUP BY (DEST_COUNTRY_NAME)
                   ORDER BY SUM(COUNT) DESC
                   LIMIT 5"""
                   )
sqlMax.show()

In [ ]:
#movendo o resultado acima para o dataframe
from pyspark.sql.functions import desc

flightData15\
    .groupBy("DEST_COUNTRY_NAME")\
    .sum("Count")\
    .withColumnRenamed("sum(Count)", "destination_total")\
    .sort(desc("destination_total"))\
    .limit(5)\
    .show()

In [ ]:
flightData15\
    .groupby("DEST_COUNTRY_NAME")\
        .sum("Count")\
            .withColumnRenamed("sum(Count)", "destination_total")\
                .sort(desc("destination_total"))\
                    .limit(5)\
                        .show()
            

In [ ]:
#finalizando
flightData15\
    .groupBy("DEST_COUNTRY_NAME")\
    .sum("Count")\
    .withColumnRenamed("sum(Count)","destination_total")\
    .sort(desc("destination_total"))\
    .limit(5)\
    .explain()

In [ ]:
from pyspark.sql.functions import avg
#esse código diz ao Spark O QUE FAZER. O que é muito melhor do que na construção com RDD.
#create a dataframe 

dataDF = spark.createDataFrame([("brooke", 20), ("Denny", 31), ("Jules", 30), 
                                ("TD", 35), ("Brooke", 25)], ["name", "age"])

#agrupando o nome, a idade e calculando a média
avgDF = dataDF.groupBy("name").agg(avg("age"))
avgDF.show()

In [ ]:
staticDataFrame = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("Spark-The-Definitive-Guide/data/retail-data/by-day/*.csv")

staticDataFrame.createOrReplaceGlobalTempView("retail_data")
staticSchema = staticDataFrame.schema

In [ ]:
staticDataFrame\
    .selectExpr("CustomerId",
                "(UnitPrice * Quantity) as TotalCost",
                "InvoiceDate")\
    .groupby(
        col("CustomerId"), window(col("InvoiceDate"),"1 Day"))\
    .sum("TotalCost")\
    .show(10)

In [ ]:
#leitura por streaming
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilePerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/Spark-The-Definitive-Guide/data/reatil-data/by-day/*.csv")
    
streamingDataFrame.isStreaming

In [ ]:
#criando uma regra de negócio de compra por hora
#utilizando streaming
#como não tem ação, isso é só uma transformação

purchaseByCustomerPerHour = streamingDataFrame\
    .selectExpr(
        "CustomerId",
        "(UnitPrice * Quantity) as TotalCost",
        "InvoiceDate")\
    .groupBy(
        col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
    .sum("TotalCost")

In [6]:
#o nome atribuído nas funções da variável devem ser minúsculos
#no sparksession, só pode haver mais de uma query com o mesmo nome
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_Purchase")\
    .outputMode("complete")\
    .start()
    

NameError: name 'purchaseByCustomerPerHour' is not defined